In [41]:
import shapely
import geopandas as gpd
import pandas as pd
import numpy as np
import math
# import matpltolib.pyplot as plt
# import seaborn as sns

## County Import

U.S. County shapefiles downloaded from U.S. Census at their TIGER/Line Shapefile portal [here](https://www.census.gov/cgi-bin/geo/shapefiles/index.php). We used the most recent shapefiles updated in 2019 and selected Counties (and equivalent) as our layer. A table of FIP section codes by county per 2018 can be found [here](https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html).

This project focuses on the contiguous 48 states, therefore we drop entries for the states of Alaska and Hawaii as well as U.S. island territories such as American Samoa, Guam, and Peurto Rice.

In [118]:
counties = gpd.read_file('../project_4_data/us_counties/tl_2019_us_county.shp')
counties.columns = counties.columns.str.lower()
counties.rename(columns = {'name':'county_name'}, inplace=True)

# Subset state counties
counties = counties.loc[~counties['statefp'].isin(['02','15','60','66','69','72','78']), :]

# Keep only FIP, name, and geometry
counties = counties.loc[:,counties.columns.isin(['geoid', 'county_name', 'geometry'])]
counties.shape

## Hurricanes

https://www.arcgis.com/home/item.html?id=9a2eb03d2b0e43df9065843d0e3efd09
https://www.nhc.noaa.gov/data/#hurdat

https://www.climate.gov/maps-data/dataset/historical-hurricane-tracks-gis-map-viewer
https://www.ncdc.noaa.gov/ibtracs/pdf/IBTrACS_v04_column_documentation.pdf

Data: https://www.ncdc.noaa.gov/ibtracs/index.php?name=ib-v4-access
Track individual hurricanes [here](https://coast.noaa.gov/hurricanes/#map=4/32/-80)

Compare with project [here](https://www.arcgis.com/apps/Cascade/index.html?appid=8f6013fdba6445e9a8732ff6cab9cd1a).

In [165]:
hurricanes = gpd.read_file('../project_4_data/IBTrACS/IBTrACS.since1980.list.v04r00.lines.shp')
hurricanes.columns = hurricanes.columns.str.lower()

# Filter for north American storms
hurricanes = hurricanes.loc[hurricanes['basin']=='NA' , : ]

# Filter for hurricanes only
hurricanes = hurricanes.loc[hurricanes['usa_status'].isin(['HU','HR'])]

# Filter for data approaching or after landfall 
hurricanes = hurricanes.loc[hurricanes['landfall']==0 , : ]

# Filter for hurricanes greater than cat 1 
hurricanes = hurricanes.loc[hurricanes['usa_sshs'] > 0 , : ]

# Keep only path geometry, strength category, and radii variables
hurricanes = hurricanes.loc[:,['sid','season','usa_sshs','usa_rmw',
                               'usa_r64_ne','usa_r64_nw', 'usa_r64_sw', 'usa_r64_se',
                               'geometry']]

print(f"Our dataset contains")
print(f"Unique Hurricanes: {hurricanes.loc[hurricanes['usa_sshs'] > 0 , 'sid' ].drop_duplicates().shape[0]} ")
print(f"Unique Hurricanes > Cat 3: {hurricanes.loc[hurricanes['usa_sshs'] > 2 , 'sid' ].drop_duplicates().shape[0]} ")


Our dataset contains
Unique Hurricanes: 100 
Unique Hurricanes > Cat 3: 37 


### Consistency Check

In [ ]:
counties = counties.to_crs('epsg:4326')
counties.crs == hurricanes.crs

## Direct Impacts

In [ ]:
h_merge = gpd.sjoin(counties, hurricanes, how='left', op='intersects')

# Remove duplicates and take maximum strength of hurricane in county
hurr_events = h_merge.loc[:,['geoid', 'sid', 'usa_sshs']].drop_duplicates()
hurr_events = hurr_events.groupby(['geoid', 'sid']).max().reset_index()

# Tabulate hurricanes per county by category strength 
hurr_events = hurr_events.pivot_table(index='geoid', columns='usa_sshs', values='sid', aggfunc = 'count').reset_index()
hurr_events.columns = ['geoid', 'hcount_cat2', 'hcount_cat3', 'hcount_cat4', 'hcount_cat5']
#hurr_events.replace({np.nan:0}, inplace=True)
hurr_events['hcount_all'] = hurr_events.drop('geoid', axis=1).sum(axis=1)

hurr_summary = counties.merge(right=hurr_events, how='left', on='geoid')
hurr_summary.head()

In [ ]:
hurr_summary.to_file("../project_4_new_data/hurricane_layer/hurricane_impacts.shp")

## Generate Area Effects after 2004

In [ ]:
# Filter for hurricanes since 2004
hurricanes = hurricanes.loc[hurricanes['season'] > 2004 , : ]

# Simplify path to central point for that time interval
hurricanes['center'] = hurricanes['geometry'].centroid

# Estimate circular  region of maximum strength 
hurricanes['core_geometry'] = hurricanes['geometry'].centroid.buffer(hurricanes['usa_rmw']/60)

# Estimate circular region of minimum cat 1 stregnth where possible
nomiss = hurricanes.loc[(hurricanes[ 'usa_sshs'] > 2) & hurricanes['usa_r64_sw'].notnull(),:]
spread = (nomiss['usa_r64_ne'] + nomiss['usa_r64_nw'] + nomiss['usa_r64_se'] + nomiss['usa_r64_sw'])/4
nomiss['spread_geometry'] = nomiss['geometry'].centroid.buffer(spread/60)
hurr_span = nomiss[['spread_geometry']]

hurricanes = hurricanes.merge(hurr_span, left_index=True, right_index=True)

hurricanes.drop(columns=['season','usa_rmw', 
                         'usa_r64_ne', 'usa_r64_nw', 
                         'usa_r64_sw', 'usa_r64_se'], inplace=True)

### Counties Tally Cat 1 or Above

In [159]:
cat1_info = hurricanes.loc[:,['sid', 'spread_geometry']]
cat1_info.rename(columns = {'spread_geometry':'geometry'}, inplace=True)
cat1_merge = gpd.sjoin(counties, cat1_info, how='left', op='intersects')
cat1_summary = cat1_merge.groupby(cat1_merge['geoid'])[['sid']].nunique().reset_index()
cat1_summary.rename(columns={'sid':'affected_count'}, inplace=True)

### Counties Tally Cat 3 or Above

In [162]:
cat3_info = hurricanes.loc[:,['sid', 'core_geometry']]
cat3_info.rename(columns = {'core_geometry':'geometry'}, inplace=True)
cat3_merge = gpd.sjoin(counties, cat3_info, how='left', op='intersects')
cat3_summary = cat3_merge.groupby(cat1_merge['geoid'])[['sid']].nunique().reset_index()
cat3_summary.rename(columns={'sid':'cat3_above_count'}, inplace=True)

## Storm Surge Inundation Hazard

NOAA National Hurricane Center SLOSH Model Hazards [here](https://www.nhc.noaa.gov/nationalsurge/).